In [337]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(6) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(6) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,50))).astype(float)

random.seed(6) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(6) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(6) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom

df_tdkom.columns = [("D"+str(i)) for i in range(1,51)]

dk = df_tdkom.transpose()
costs_t = dk.to_dict()
colsn = sorted(df_demand.keys())

# print(supply['S1'])
# break

costs1=copy.deepcopy(costs_t)
costs2=copy.deepcopy(costs_t)
# costs3=copy.deepcopy(costs_t)
# for i in df_supply:
#     mi=min(costs_t[i].values())
#     # print(costs[i])
#     # print(mi)
#     for j in costs2[i]:
#         costs2[i][j]-=mi
# # print(costs2)
# for i in df_demand :
#     mi=10000
#     for j in df_supply:
#         if costs_t[j][i]<mi :
#             mi=costs_t[j][i]
#     for j in df_supply:
#         costs3[j][i]=costs3[j][i]-mi
# # print(costs3)

# for i in df_demand:
#     for j in df_supply:
#         costs_t[j][i]= costs2[j][i]+costs3[j][i]
# # print(costs_t)


res = dict((k, defaultdict(int)) for k in costs_t)

g = {}
for x in df_supply:
    # print(x)
    # print(costs_t[x])
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

while g:
    d = {}
    # print(df_supply,dic)
    for x in df_demand:
        d[x] = ((costs_t[g[x][1]][x] - costs_t[g[x][0]][x])/df_demand[x])*costs_t[g[x][0]][x] if len(g[x]) > 1 else (costs_t[g[x][0]][x]/df_demand[x])*costs_t[g[x][0]][x]
    s = {}
    for x in df_supply:
        s[x] = ((costs_t[x][g[x][1]] - costs_t[x][g[x][0]])/df_supply[x])*costs_t[x][g[x][0]]  if len(g[x]) > 1 else (costs_t[x][g[x][0]]/df_supply[x])*costs_t[x][g[x][0]]

    f = max(d, key=lambda n: d[n])
    t = max(s, key=lambda n: s[n])


    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


    v = min(df_supply[f], df_demand[t])
    # print(f,t)
    # print(v)
    res[f][t] += v
    df_demand[t] -= v

    if df_demand[t] == 0:
        for k, n in df_supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del df_demand[t]
    df_supply[f] -= v
    if df_supply[f] == 0:
        for k, n in df_demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del df_supply[f]

In [338]:
# print("G",g)
cost = 0
# cols = sorted(df_demand.keys())
# print(costs)
for g in sorted(costs1):
    # print (g, " ",)
    # print("S")
    for n in colsn:
        y = res[g][n]
        # print("YESS",y)
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")

In [339]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D41,D42,D43,D44,D45,D46,D47,D48,D49,D50
0,622.384763,298.512752,565.424501,102.135964,127.333379,443.134186,395.670762,345.760381,278.061286,464.430937,...,530.065736,519.717692,340.683487,687.516575,327.575777,299.743887,506.533402,127.912455,325.379958,516.455920
1,408.129975,215.800911,160.952869,263.287678,195.253109,389.509944,160.038551,231.725465,419.143860,346.642317,...,139.714502,381.479794,183.012522,306.727863,76.502550,205.481745,438.148858,300.563559,310.067350,213.018682
2,550.853162,254.220993,155.977383,380.922322,208.278263,515.632913,184.270809,524.919562,410.922667,124.908225,...,357.247416,435.345530,629.738927,304.288892,186.615585,610.579699,411.050867,609.330064,616.541350,449.202533
3,111.767709,404.481223,209.329362,139.198018,242.381517,193.939126,462.313391,559.099961,94.202704,226.777221,...,196.460414,463.179610,326.527927,167.118635,169.694778,537.249389,96.547862,392.854274,408.465656,400.960364
4,337.382847,423.481915,421.946617,338.972235,134.242087,391.625058,200.839783,443.734663,313.097354,134.008946,...,370.892640,303.211750,441.062944,389.671758,435.323540,127.795488,435.120230,464.777924,325.314140,66.704553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5254,650.348798,632.464557,513.563066,466.017677,174.440438,588.578447,397.534448,147.432319,137.502038,627.326778,...,414.305267,551.861165,393.287936,417.709158,244.087292,213.008736,292.891533,478.580795,218.797348,372.789825
5255,429.986043,417.874297,172.732180,159.062894,110.682045,243.721083,421.771968,258.463930,277.390348,256.614483,...,147.567425,377.006859,139.517181,405.266636,224.593523,142.941522,383.718151,443.286775,299.629693,439.988640
5256,193.411362,186.563704,398.122858,515.569929,265.979522,661.031266,90.784166,158.812984,411.152081,376.042572,...,565.821659,277.652666,530.407159,695.677180,639.926342,206.164530,110.335736,96.746563,172.011077,480.298926
5257,408.209481,247.181356,288.449286,259.303033,441.556139,394.612932,165.111513,107.928619,272.578412,496.662570,...,345.697887,140.299349,189.588518,184.746528,300.847404,480.004870,154.488021,247.937149,332.616233,127.002599


In [340]:
cost

546679.9075039279